In [71]:
import json
import io
import os
import wave
from pydub import AudioSegment
from vosk import Model, KaldiRecognizer, SetLogLevel

from mp3.common import (initialize_model_tokenizer_SENT, analyze_sentiment)

SetLogLevel(-1)

VOSK_MODEL = "vosk/vosk-model-small-en-us-0.15"
SENT_MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
SENT_TKNZR =  "cardiffnlp/twitter-roberta-base-sentiment"

In [88]:
def initialize_model_VOSK(model_path: str) -> Model:
    
    model = Model(model_path)

    return model

def mp3_to_wav(source:str, skip: int = 0, cut_at: int =30):
    '''
    based from 
    towardsdatascience.com/transcribe-large-audio-files-offline-with-vosk-a77ee8f7aa28
    '''
    
    if skip >= cut_at:
        assert False, "'cut_at' vakue must be greater thank 'skip' value"

    sound = AudioSegment.from_mp3(source) # load source
    sound = sound.set_channels(1) # mono
    sound = sound.set_frame_rate(16000) # 16000Hz
    
    sound = sound[skip*1000:cut_at*1000]
    # output_path = os.path.splitext(source)[0]+".wav"
    
    audio = io.BytesIO()
    sound.export(audio, format="wav")
    
    return audio

# def mp3_to_wav(source:str, skip: int = 0, cut_at: int = 60):
#     '''
#     based from 
#     towardsdatascience.com/transcribe-large-audio-files-offline-with-vosk-a77ee8f7aa28
#     '''
    
#     if skip >= cut_at:
#         assert False, "'cut_at' vakue must be greater thank 'skip' value"

#     sound = AudioSegment.from_mp3(source) # load source
#     sound = sound.set_channels(1) # mono
#     # sound = sound.set_frame_rate(16000) # 16000Hz
    
#     sound = sound[skip*1000:cut_at*1000]
#     output_path = os.path.splitext(source)[0]+".wav"
#     sound.export(output_path, format="wav")
    
#     return output_path

In [83]:
model = initialize_model_VOSK(model_path = VOSK_MODEL)

sent_model, sent_tokenizer = initialize_model_tokenizer_SENT(model_name=SENT_MODEL,
                                                               tokenizer_name=SENT_TKNZR)

In [96]:
# wav_path = mp3_to_wav('Puppy.mp3', skip=0, cut_at=60)
# # wav = mp3_to_wav('Puppy.mp3', skip=0, cut_at=60)
# wf = wave.open('Puppy.wav', "rb")

wf = mp3_to_wav('Puppy.mp3', skip=0, cut_at=60)
wf = wave.open(wf, "rb")

rec = KaldiRecognizer(model, wf.getframerate())
rec.SetWords(True)
rec.SetPartialWords(True)

while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        pass
    else:
        pass

result = json.loads(rec.FinalResult())

sentence = result['text']
sentiment = analyze_sentiment(sentence=sentence,
                              model=sent_model,
                              tokenizer=sent_tokenizer)

print(sentiment)

[0.0123186  0.66499496 0.32268637]
